
<h1><center> PPOL 6801 Text as Data <br><br> 
<font color='grey'> LLMs: Outsourcing Text-as-Data Tasks  <br><br>
Tiago Ventura </center> <h1> 

---

### Outsourcing Text-as-Data Tasks to Generative Text-Based Models: GPT's API

As you know, ChatGPT is an large language model (as we just saw) developed by OpenAI, based on the GPT architecture. The model was trained on a word-prediction task and it has blown the world by its capacity to engage in conversational interactions.

You should be familiar with interacting with the ChatGPT tool to solve a variety of tasks. Here I will show you how to do that at scale, by using prompts to interact with the model via Open AI API. 

The whole process requires us to have access to the Open AI API which allow us to query continously the GPT models. Notice, this is not free. You pay for every query. In general, for small tasks, it is not super expensive. However, for tasks with millions of predictions, it can get expensive. 



## Tasks and Prompts

Before we try to replicate the tasks behind the papers we read in class, let's see some simple tasks we can ask GPT models to perform. 

In [8]:
#!pip install openai

In [9]:
# load api key
# load library to get environmental files
import os
from dotenv import load_dotenv
import requests 


# load keys from  environmental var
load_dotenv() # .env file in cwd
gpt_key = os.environ.get("gpt") 

In [10]:
# simple query

# define headers
headers = {
        "Authorization": f"Bearer {gpt_key}",
        "Content-Type": "application/json",
    }

# define gpt model
question = "Please, tell me more about the Data Science and Public Policy Program at Georgetown's McCourt School"

data = {
        "model": "gpt-4",
        "temperature": 0,
        "messages": [{"role": "user", "content": question}]
    }



# send a post request
response = requests.post("https://api.openai.com/v1/chat/completions", 
                             json=data, 
                             headers=headers)
# convert to json
response_json = response.json()

In [11]:
response_json['choices'][0]['message']['content'].strip()

"The Data Science and Public Policy (DSPP) program at Georgetown University's McCourt School of Public Policy is a unique program that combines traditional public policy studies with cutting-edge technical skills in data science. The program is designed to equip students with the ability to use data-driven approaches to solve complex policy issues.\n\nThe curriculum of the DSPP program includes courses in statistics, computer science, and public policy. Students learn how to collect, analyze, and interpret large datasets, and how to use these skills to inform policy decisions. They also learn about the ethical and legal implications of using data in this way.\n\nThe program is designed for students who have a strong interest in public policy and a desire to use data science to make a positive impact on society. Graduates of the program are prepared for careers in government, non-profit organizations, and the private sector, where they can use their skills to inform policy decisions and

## Sentiment Classification

In Rathje et. al., we saw the use of GPT models for sentiment classification using zero-shot prompts. 

This is a super simple task. Let's see some code below on how to go about it.  

Get the data here: 

In [14]:
# Function to interact with the ChatGPT API
def hey_chatGPT(question_text, api_key):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    data = {
        "model": "gpt-4",
        "temperature": 0,
        "messages": [{"role": "user", "content": question_text}]
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", 
                             json=data, 
                             headers=headers, timeout=5)
    
    response_json = response.json()
    return response_json['choices'][0]['message']['content'].strip()

In [16]:
import pandas as pd
# let's open some twitter data
pd_test = pd.read_csv("incivility.csv")

# sample
pd_test = pd_test.sample(n=10).reset_index()

# see
pd_test.head()

,index,comment_id,comment_likes_count,comment_message,attacks
0,2496,1162593253768645_1162795933748377,4,You're a great rep... of the gun companies pay...,1
1,1899,1098565546882910_1098608580211940,5,"As with background checks, and as with attempt...",0
2,409,1185839031442312_1186407618052120,1,SHAMEFUL vote on Country of Origin Labeling Am...,1
3,952,1015212438503006_1015933865097530,0,"Yes, Todd, we appreciated the meeting but not ...",0
4,385,10153172973824110_10153173135759110,7,California: mismanaged since the first Brown a...,0


In [17]:
import time
output = []
# Run a loop over your dataset of reviews and prompt ChatGPT
for i in range(len(pd_test)):
    try: 
        print(i)
        question = "Is the sentiment of this text positive, neutral, or negative? \
        Answer only with a number: 1 if positive, 0 if neutral and -1 if negative. \
        Here is the text: "
        text = pd_test.loc[i, "comment_message"]
        full_question = question + str(text)
        output.append(hey_chatGPT(full_question, gpt_key))
    except:
        output.append(np.nan)

0
1
2
3
4
5
6
7
8
9


In [18]:
# save the output
pd_test["sentiment"]= output

In [19]:
# see
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(pd_test[["comment_message", "sentiment"]])


                                     comment_message sentiment
0  You're a great rep... of the gun companies pay...        -1
1  As with background checks, and as with attempt...        -1
2  SHAMEFUL vote on Country of Origin Labeling Am...        -1
3  Yes, Todd, we appreciated the meeting but not ...        -1
4  California: mismanaged since the first Brown a...        -1
5  The more the words, the less the meaning. Phas...        -1
6  Senator, please learn the difference between "...        -1
7                 Bill you can't get it done, retire        -1
8  Do the right and honest thing--represent your ...         1
9                Unbelievable level of incompetence!        -1


In [20]:
for i in range(len(pd_test)):
    print("Text:" + pd_test["comment_message"][i] + " \nSentiment: " + pd_test["sentiment"][i])

Text:You're a great rep... of the gun companies paying you. 
Sentiment: -1
Text:As with background checks, and as with attempting to sabotage State Dept negotiations on nuclear disarmament, your position here defies logic, and also is contrary to the overwhelming opinion of your constituents. We elected you to do your job. We elected Pres. Obama to do his job. Neither one of you should stop doing your job until the next time people decide who gets to do your job. We did not elect you to serve for 5 years, and then sit on your hands for 1 year awaiting the next time the public could voice their opinion. Nor did we re-elect Pres Obama to serve for 3 years and then sit on his hands. Do your job. 
Sentiment: -1
Text:SHAMEFUL vote on Country of Origin Labeling Amendments Act of 2015, Congresswoman. You voted with the House republicans to take away an American consumer's right to know what the hell we're feeding our kids and for that vote, alone, you should be kicked out of office.  You have

## Scaling via pair-wise comparison

Now let's see how we can use GPT to do pairwise comparison. Notice, we saw in the paper that pairwise comparisons can be used as input for scaling models of ideology. But, this type of labeled data can be used for many different tasks, for example, readability and sophistication scores, as we saw earlier in the semester. 

Together with Lisa Signh and Leticia Bode, we are actually using a similar approach, but we human labelling, to understand levels of hummaness of social media content in the AI-Era. Next year, you can email me and I can show you some results!

The code below was actually provided by Patrick Wu. So thanks to him!

In [21]:
# bring soma
import pandas as pd
import numpy as np
import os
import time
from openai import OpenAI
from itertools import combinations
from random import sample, choices
import random
import re
from tqdm import tqdm
from joblib import delayed, Parallel

In [22]:
# create a client to interact with the API
client = OpenAI(
    # This is the default and can be omitted
    api_key=gpt_key,
)

In [23]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4",
)


In [24]:
'''
p: the prompt
system_prompt: the system prompt. The default is what is used on the ChatGPT's web interface.
temp: temperature parameter. 1.0 is the default (for GPT-3.5, temperature ranges from 0 to 2.0)
request_timeout: the amount of time, in seconds, to timeout the function.
'''
def prompting_openai_comparison(p,
                                system_prompt='You are ChatGPT, a large language model trained by OpenAI, based on the GPT-3.5 architecture.\nKnowledge cutoff: 2021-09\nCurrent date: 2023-10-28',
                                temp=1.0):
  # times used to sleep; these values approximate exponential backoff
    sleepy_times = [1, 2]

    for i in range(len(sleepy_times)):
        try:
            response = client.chat.completions.create(model="gpt-3.5-turbo",
                                              messages=[{"role": "system", 
                                                         "content": system_prompt},
                                                        {"role": "user", 
                                                         "content": p}],
                                              temperature=0)
            break
        except:
          # if OpenAI's API returns an error, this lets you know and backs off for the set time, determined using the sleepy_times list
          print('uh oh, ' + str(sleepy_times[i]))
          time.sleep(sleepy_times[i])
    return response

In [28]:
# get a list of S116 members
import urllib.request
url = "https://voteview.com/static/data/out/members/S116_members.csv"
urllib.request.urlretrieve(url, "S116_members.csv")

('S116_members.csv', <http.client.HTTPMessage at 0x16cdcf810>)

In [29]:
df = pd.read_csv('S116_members.csv')

In [30]:
# let's get an ordinary version of some members of the congress
# Add "ordinary" versions of senators' names
df['bioname_ordinary'] = ['Donald Trump',
'Doug Jones',
'Richard Shelby',
'Lisa Murkowski',
'Dan Sullivan',
'Kyrsten Sinema',
'Martha McSally',
'Mark Kelly',
'John Boozman',
'Tom Cotton',
'Kamala Harris',
'Dianne Feinstein',
'Cory Gardner',
'Michael Bennet',
'Chris Murphy',
'Richard Blumenthal',
'Tom Carper',
'Chris Coons',
'Marco Rubio',
'Rick Scott',
'Johnny Isakson',
'David Perdue',
'Kelly Loeffler',
'Mazie Hirono',
'Brian Schatz',
'Mike Crapo',
'James Risch',
'Dick Durbin',
'Tammy Duckworth',
'Todd Young',
'Mike Braun',
'Chuck Grassley',
'Joni Ernst',
'Pat Roberts',
'Jerry Moran',
'Mitch McConnell',
'Rand Paul',
'Bill Cassidy',
'John Kennedy',
'Angus King',
'Susan Collins',
'Ben Cardin',
'Chris Van Hollen',
'Ed Markey',
'Elizabeth Warren',
'Gary Peters',
'Debbie Stabenow',
'Amy Klobuchar',
'Tina Smith',
'Roger Wicker',
'Cindy Hyde-Smith',
'Roy Blunt',
'Josh Hawley',
'Steve Daines',
'Jon Tester',
'Deb Fischer',
'Ben Sasse',
'Jacky Rosen',
'Catherine Cortez Masto',
'Jeanne Shaheen',
'Maggie Hassan',
'Bob Menendez',
'Cory Booker',
'Martin Heinrich',
'Tom Udall',
'Chuck Schumer',
'Kirsten Gillibrand',
'Richard Burr',
'Thom Tillis',
'Kevin Cramer',
'John Hoeven',
'Rob Portman',
'Sherrod Brown',
'Jim Inhofe',
'James Lankford',
'Ron Wyden',
'Jeff Merkley',
'Pat Toomey',
'Bob Casey',
'Jack Reed',
'Sheldon Whitehouse',
'Tim Scott',
'Lindsey Graham',
'John Thune',
'Mike Rounds',
'Marsha Blackburn',
'Lamar Alexander',
'John Cornyn',
'Ted Cruz',
'Mike Lee',
'Mitt Romney',
'Patrick Leahy',
'Bernie Sanders',
'Mark Warner',
'Tim Kaine',
'Maria Cantwell',
'Patty Murray',
'Shelley Moore Capito',
'Joe Manchin',
'Tammy Baldwin',
'Ron Johnson',
'John Barrasso',
'Mike Enzi']

In [31]:
# Delete Donald Trump
df = df.iloc[1:,]

# sample just a few
df = df.sample(n=10).reset_index()


In [32]:
df

,index,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,...,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2,bioname_ordinary
0,3,116,Senate,40300,81,0.0,AK,200,NaN,NaN,...,0.203,-0.304,-49.06021,0.92427,623.0,12.0,NaN,0.283,-0.424,Lisa Murkowski
1,28,116,Senate,21325,21,0.0,IL,100,NaN,NaN,...,-0.344,0.061,-95.71677,0.86229,646.0,36.0,NaN,-0.376,0.208,Tammy Duckworth
2,65,116,Senate,14858,13,0.0,NY,100,NaN,NaN,...,-0.355,-0.414,-105.71518,0.85363,668.0,40.0,NaN,-0.416,-0.387,Chuck Schumer
3,91,116,Senate,14307,6,0.0,VT,100,NaN,NaN,...,-0.360,-0.121,-117.44960,0.83399,647.0,67.0,NaN,-0.279,-0.040,Patrick Leahy
4,21,116,Senate,41501,44,0.0,GA,200,NaN,NaN,...,0.562,-0.075,-38.26812,0.93689,587.0,19.0,NaN,0.474,0.156,David Perdue
5,22,116,Senate,41904,44,0.0,GA,200,NaN,NaN,...,0.555,-0.203,-18.40266,0.92180,226.0,7.0,NaN,0.556,-0.201,Kelly Loeffler
6,49,116,Senate,29534,46,0.0,MS,200,NaN,NaN,...,0.377,0.340,-27.94363,0.95897,667.0,8.0,NaN,0.364,0.196,Roger Wicker
7,17,116,Senate,40916,11,0.0,DE,100,NaN,NaN,...,-0.238,-0.217,-77.44469,0.88686,645.0,34.0,NaN,-0.210,-0.237,Chris Coons
8,40,116,Senate,49703,2,0.0,ME,200,NaN,NaN,...,0.124,-0.505,-57.26805,0.91784,668.0,16.0,NaN,0.139,-0.404,Susan Collins
9,8,116,Senate,20101,42,0.0,AR,200,NaN,NaN,...,0.427,0.338,-31.38236,0.95411,668.0,16.0,NaN,0.354,0.245,John Boozman


In [33]:
# Then get dictionaries that obtain the party and state for each senator by name
names = list(df['bioname_ordinary'])
state = list(df['state_abbrev'])
party = ['R' if j==200 else 'D' if j==100 else 'I' for j in list(df['party_code'])]

name_party_dict = {n: p for n,p in zip(names,party)}
name_state_dict = {n: s for n,s in zip(names,state)}

In [34]:
# this function samples a total number of matchups per senator. this does not mean that each senator is limited to a max of sample_size matchups
# it means each senator will appear in at least sample_size matchups
def generate_pairwise_matchups(items, sample_size=20, seed_value=42):
  random.seed(seed_value)

  if sample_size >= len(items) or sample_size < 1:
    raise ValueError("Sample size must be between 1 and one less than the total number of tweet IDs")

  all_matchups = []

  # Generate all possible pairings
  all_combinations = list(combinations(items, 2))

  for i in items:
    # Filter matchups containing the current tweet ID
    relevant_matchups = [pair for pair in all_combinations if i in pair]

    # Shuffle the matchups
    random.shuffle(relevant_matchups)

    # Sample from these matchups up to the specified sample size
    all_matchups.extend(relevant_matchups[:sample_size])

  return all_matchups

In [35]:
matchups = generate_pairwise_matchups(names, sample_size=1, seed_value=42)

In [36]:
len(matchups)

10

Here, we note the direction of comparison. We have to use liberal and conservative differently in these prompts because, when comparing two Republicans, if I prompt ChatGPT with "who is more liberal," it will often fail to answer this and reply that both senators are conservative.

In [37]:
prompts = []
comparison_direction = []

for j in matchups:
    # D vs. D
    if (name_party_dict[j[0]]=='D' or name_party_dict[j[0]]=='I') and (name_party_dict[j[1]]=='D' or name_party_dict[j[1]]=='I'):
        sent = 'Based on past voting records and statements, which senator is more liberal: ' + j[0] + ' (' + name_party_dict[j[0]] + '-' + name_state_dict[j[0]] + ') or ' + j[1] + ' (' + name_party_dict[j[1]] + '-' + name_state_dict[j[1]] + ')?'
        comparison_direction.append('liberal')
    # D vs. R
    elif (name_party_dict[j[0]]=='D' or name_party_dict[j[0]]=='I') and (name_party_dict[j[1]]=='R'):
        sent = 'Based on past voting records and statements, which senator is more liberal: ' + j[0] + ' (' + name_party_dict[j[0]] + '-' + name_state_dict[j[0]] + ') or ' + j[1] + ' (' + name_party_dict[j[1]] + '-' + name_state_dict[j[1]] + ')?'
        comparison_direction.append('liberal')
    # R vs. D
    elif (name_party_dict[j[0]]=='R') and (name_party_dict[j[1]]=='D' or name_party_dict[j[1]]=='I'):
        sent = 'Based on past voting records and statements, which senator is more liberal: ' + j[0] + ' (' + name_party_dict[j[0]] + '-' + name_state_dict[j[0]] + ') or ' + j[1] + ' (' + name_party_dict[j[1]] + '-' + name_state_dict[j[1]] + ')?'
        comparison_direction.append('liberal')
    # R vs. R
    elif (name_party_dict[j[0]]=='R') and (name_party_dict[j[1]]=='R'):
        sent = 'Based on past voting records and statements, which senator is more conservative: ' + j[0] + ' (' + name_party_dict[j[0]] + '-' + name_state_dict[j[0]] + ') or ' + j[1] + ' (' + name_party_dict[j[1]] + '-' + name_state_dict[j[1]] + ')?'
        comparison_direction.append('conservative')
    else:
        print('OH NO!')
        break
    prompts.append(sent)

In [38]:
#Set the system prompt for the pairwise comparison.
system_prompt = 'You are ChatGPT, a large language model trained by OpenAI, based on the GPT-3.5 architecture.\nKnowledge cutoff: 2021-09\nCurrent date: 2023-09-11'

In [39]:
print(prompts[0:10])

['Based on past voting records and statements, which senator is more conservative: Lisa Murkowski (R-AK) or David Perdue (R-GA)?', 'Based on past voting records and statements, which senator is more liberal: Tammy Duckworth (D-IL) or Susan Collins (R-ME)?', 'Based on past voting records and statements, which senator is more liberal: Chuck Schumer (D-NY) or Patrick Leahy (D-VT)?', 'Based on past voting records and statements, which senator is more liberal: Patrick Leahy (D-VT) or John Boozman (R-AR)?', 'Based on past voting records and statements, which senator is more conservative: David Perdue (R-GA) or Kelly Loeffler (R-GA)?', 'Based on past voting records and statements, which senator is more liberal: Chuck Schumer (D-NY) or Kelly Loeffler (R-GA)?', 'Based on past voting records and statements, which senator is more liberal: Tammy Duckworth (D-IL) or Roger Wicker (R-MS)?', 'Based on past voting records and statements, which senator is more liberal: Roger Wicker (R-MS) or Chris Coons

Now we're finally ready to run the pairwise comparisons. We run it in parallel because it takes a very long time to run if you iterate one prompt at a time.

In [40]:
# create a container
comparison_results = []

# iterate
for p in prompts:
    results = prompting_openai_comparison(p, system_prompt, 1.0)
    comparison_results.append(results)

In [41]:
# let's look at it 
comparison_results[0]

ChatCompletion(id='chatcmpl-CfbqjBLhT8cflQQyR7y2J9sUjqmhS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="As of my last knowledge update in September 2021, both Lisa Murkowski and David Perdue were considered to be Republicans, but they have different conservative leanings.\n\nLisa Murkowski, a Senator from Alaska, has been known to be more moderate or centrist within the Republican Party. She has taken positions that sometimes diverge from the more conservative elements of the party, particularly on issues like healthcare and the environment.\n\nDavid Perdue, a former Senator from Georgia, was generally considered to be more conservative in his voting record and policy positions. He aligned more closely with the conservative wing of the Republican Party on issues such as taxes, healthcare, and immigration.\n\nIt's important to note that political positions can evolve, and new information may have emerged since my last update. I re

In [42]:
# Extract the text answer from ChatGPT responses
def get_text_from_chatgpt(responses):
  return [responses[i].choices[0].message.content for i in range(len(responses))]

In [43]:
comparisons_text = get_text_from_chatgpt(comparison_results)

In [44]:
# This is what a pairwise comparison looks like.
print(comparisons_text[0])

As of my last knowledge update in September 2021, both Lisa Murkowski and David Perdue were considered to be Republicans, but they have different conservative leanings.

Lisa Murkowski, a Senator from Alaska, has been known to be more moderate or centrist within the Republican Party. She has taken positions that sometimes diverge from the more conservative elements of the party, particularly on issues like healthcare and the environment.

David Perdue, a former Senator from Georgia, was generally considered to be more conservative in his voting record and policy positions. He aligned more closely with the conservative wing of the Republican Party on issues such as taxes, healthcare, and immigration.

It's important to note that political positions can evolve, and new information may have emerged since my last update. I recommend checking more recent sources to get the most up-to-date information on the political stances of Lisa Murkowski and David Perdue.


Now we need to extract the answers from our comparisons. We'll append our answers from before and ask ChatGPT to extract the answer.

In [45]:
extracting_answer_prompt = []

for i in range(len(comparisons_text)):
    if comparison_direction[i]=='liberal':
        sent = 'Text: "' + comparisons_text[i] + '"\n\nIn the above Text, who is described to be the more liberal, more progressive, or less conservative senator: ' + matchups[i][0] + ' or ' + matchups[i][1] + '? Return only the full name without party affiliation or state information. If one senator is described as more conservative, return the other senator\'s name. If one senator is described as more moderate, return the other senator\'s name. If neither senators are described to be more liberal, more progressive, less conservative, more conservative, or more moderate, reply with "Tie."'
    elif comparison_direction[i]=='conservative':
        sent = 'Text: "' + comparisons_text[i] + '"\n\nIn the above Text, who is described to be the more conservative or less liberal senator: ' + matchups[i][0] + ' or ' + matchups[i][1] + '? Return only the full name without party affiliation or state information. If one senator is described as more liberal, return the other senator\'s name. If one senator is described as more moderate, return the other senator\'s name. If neither senators are described to be more conservative, less liberal, more liberal, or more moderate, reply with "Tie."'
    extracting_answer_prompt.append(sent)

In [46]:
system_prompt_extraction = 'You are reading a Text and extracting information from it according to the prompt. Follow the directions exactly.'

In [47]:
# let's see this!
extracting_answer_prompt[0]

'Text: "As of my last knowledge update in September 2021, both Lisa Murkowski and David Perdue were considered to be Republicans, but they have different conservative leanings.\n\nLisa Murkowski, a Senator from Alaska, has been known to be more moderate or centrist within the Republican Party. She has taken positions that sometimes diverge from the more conservative elements of the party, particularly on issues like healthcare and the environment.\n\nDavid Perdue, a former Senator from Georgia, was generally considered to be more conservative in his voting record and policy positions. He aligned more closely with the conservative wing of the Republican Party on issues such as taxes, healthcare, and immigration.\n\nIt\'s important to note that political positions can evolve, and new information may have emerged since my last update. I recommend checking more recent sources to get the most up-to-date information on the political stances of Lisa Murkowski and David Perdue."\n\nIn the abov

In [48]:
# create a container
extraction = []

# iterate
for p in extracting_answer_prompt:
    results = prompting_openai_comparison(p, system_prompt, 1.0)
    extraction.append(results)

In [49]:
extraction_text = get_text_from_chatgpt(extraction)

In [50]:
extraction_text

['David Perdue',
 'Tammy Duckworth',
 'Patrick Leahy',
 'Patrick Leahy',
 'David Perdue',
 'Chuck Schumer',
 'Tammy Duckworth',
 'Chris Coons',
 'Susan Collins',
 'John Boozman']

In [51]:
# this function simply removes the period at the sentences
def remove_period(sentence):
    if sentence.endswith('.'):
        sentence = sentence[:-1]
    return sentence

# this function simply removes the 'Senator ' prefix. For example, it returns "Dianne Feinstein" if the input text is "Senator Dianne Feinstein"
def remove_senator_prefix(input_string):
    if input_string.startswith("Senator "):
        return input_string[8:]
    else:
        return input_string

In [52]:
extraction_text = [remove_period(t) for t in extraction_text]
extraction_text = [remove_senator_prefix(t) for t in extraction_text]

In [53]:
print(extraction_text[0])

David Perdue


We then use a function to check that every extraction was correct. Sometimes it will still not correctly extract the answer, which means we have to step in and manually fix it. If the function prints nothing, great!

This step will make the final dataframe with the resultant matchups.

In [54]:
def make_final_df(matchups, chatgpt_answers, final_answers, comparison_direction):
    sen1 = [j[0] for j in matchups]
    sen2 = [j[1] for j in matchups]

    matchup_results = pd.DataFrame({'matchup': matchups,
                                    'senator1': sen1,
                                    'senator2': sen2,
                                    'chatgpt_response': chatgpt_answers,
                                    'final_answers': final_answers,
                                    'comparison_direction': comparison_direction})

    opposite = []
    sen1_win = []
    sen2_win = []

    for i in range(len(matchup_results['matchup'])):
        if matchup_results['comparison_direction'][i]=='liberal':
            if matchup_results['final_answers'][i]==matchup_results['senator1'][i]:
                sen1_win.append(0.0)
                sen2_win.append(1.0)
            elif matchup_results['final_answers'][i]==matchup_results['senator2'][i]:
                sen1_win.append(1.0)
                sen2_win.append(0.0)
            elif matchup_results['final_answers'][i]=='Tie':
                sen1_win.append(0.5)
                sen2_win.append(0.5)
        elif matchup_results['comparison_direction'][i]=='conservative':
            if matchup_results['final_answers'][i]==matchup_results['senator1'][i]:
                sen1_win.append(1.0)
                sen2_win.append(0.0)
            elif matchup_results['final_answers'][i]==matchup_results['senator2'][i]:
                sen1_win.append(0.0)
                sen2_win.append(1.0)
            elif matchup_results['final_answers'][i]=='Tie':
                sen1_win.append(0.5)
                sen2_win.append(0.5)
        else:
            print(str(i) + ' is a defective outcome')

    matchup_results['win1'] = sen1_win
    matchup_results['win2'] = sen2_win

    return matchup_results

In [55]:
final_df = make_final_df(matchups=matchups,
                         chatgpt_answers=comparisons_text,
                         final_answers=extraction_text,
                         comparison_direction=comparison_direction)

In [56]:
final_df

,matchup,senator1,senator2,chatgpt_response,final_answers,comparison_direction,win1,win2
0,"(Lisa Murkowski, David Perdue)",Lisa Murkowski,David Perdue,As of my last knowledge update in September 20...,David Perdue,conservative,0.0,1.0
1,"(Tammy Duckworth, Susan Collins)",Tammy Duckworth,Susan Collins,"Tammy Duckworth, a Democrat from Illinois, is ...",Tammy Duckworth,liberal,0.0,1.0
2,"(Chuck Schumer, Patrick Leahy)",Chuck Schumer,Patrick Leahy,As of my last knowledge update in September 20...,Patrick Leahy,liberal,1.0,0.0
3,"(Patrick Leahy, John Boozman)",Patrick Leahy,John Boozman,"Patrick Leahy, a Democrat from Vermont, is gen...",Patrick Leahy,liberal,0.0,1.0
4,"(David Perdue, Kelly Loeffler)",David Perdue,Kelly Loeffler,As of my last knowledge update in September 20...,David Perdue,conservative,1.0,0.0
5,"(Chuck Schumer, Kelly Loeffler)",Chuck Schumer,Kelly Loeffler,As of my last knowledge update in September 20...,Chuck Schumer,liberal,0.0,1.0
6,"(Tammy Duckworth, Roger Wicker)",Tammy Duckworth,Roger Wicker,As of my last knowledge update in September 20...,Tammy Duckworth,liberal,0.0,1.0
7,"(Roger Wicker, Chris Coons)",Roger Wicker,Chris Coons,As of my last knowledge update in September 20...,Chris Coons,liberal,1.0,0.0
8,"(Lisa Murkowski, Susan Collins)",Lisa Murkowski,Susan Collins,As of my last knowledge update in September 20...,Susan Collins,conservative,0.0,1.0
9,"(Susan Collins, John Boozman)",Susan Collins,John Boozman,As of my last knowledge update in September 20...,John Boozman,conservative,0.0,1.0


From here would just need to go the R to do the Bradley Terry model. Happy to share code about this as well, but that would be too much for today!

## Generating survey responses

In [63]:
# Function to interact with the ChatGPT API
def survey_chatGPT(profile, prompt, api_key):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    data = {
        "model": "gpt-4",
        "temperature": 0.2,
        "messages": [{"role": "system", 
                      "content": profile}, 
                    {"role":"user", 
                    "content":prompt}]
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", 
                             json=data, 
                             headers=headers, timeout=5)
    
    response_json = response.json()
    return response_json

In [64]:
def gen_profile(age, race, gender, educ, inc, pid):
    profile = "You are a " + str(age) + " year old " + race + " "+ gender + " with a " + educ + ", earning $" + inc + " per year . " + "You are a registered " + pid + " living in the USA in 2019. "
    return profile

In [65]:
# test one profile
profile = gen_profile(18, "latino", "female", "post-graduate", "100,000", "Democrat")
profile

'You are a 18 year old latino female with a post-graduate, earning $100,000 per year . You are a registered Democrat living in the USA in 2019. '

In [66]:
prompt = "Provide responses from this person's perspective.\n"\
         "Use only knowledge about politics that they would have.\n"\
         "Format the output as a csv table with the following format:\n"\
         "group, thermometer\n"\
         "The following questions ask about individuals' feelings "\
         "toward different groups.\n"\
         "Responses should be given on a scale from 0 (meaning cold "\
         "feelings) to 100 (meaning warm feelings).\n"\
         "Ratings between 50 degrees and 100 degrees mean that\n"\
         "you feel favorable and warm toward the group. Ratings "\
         "between 0\n"\
         "degrees and 50 degrees mean that you don't feel "\
         "favorable toward\n"\
         "the group and that you don't care too much for that "\
         "group. You\n"\
         "would rate the group at the 50 degree mark if you don't feel\n"\
         "particularly warm or cold toward the group.\n"\
         "How do you feel toward the following groups?\n"\
         "The Democratic Party?\n"\
         "The Republican Party?\n"\
         "Democrats?\n"\
         "Republicans?\n"\
         "Black Americans?\n"\
         "White Americans?\n"\
         "Hispanic Americans?\n"\
         "Asian Americans?\n"\
         "Muslims?\n"\
         "Christians?\n"\
         "Immigrants?\n"\
         "Gays and Lesbians?\n"\
         "Jews?\n"\
         "Liberals?\n"\
         "Conservatives?\n"\
         "Women?\n"

In [67]:
# get an output
output = survey_chatGPT(profile, prompt, gpt_key)
print(output)

{'id': 'chatcmpl-Cfbs61ySvgIQIbsRPWaxIiV7vbmXT', 'object': 'chat.completion', 'created': 1764033062, 'model': 'gpt-4-0613', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '"Democratic Party,85\nRepublican Party,35\nDemocrats,85\nRepublicans,35\nBlack Americans,80\nWhite Americans,75\nHispanic Americans,90\nAsian Americans,80\nMuslims,75\nChristians,70\nImmigrants,85\nGays and Lesbians,90\nJews,75\nLiberals,85\nConservatives,35\nWomen,95"', 'refusal': None, 'annotations': []}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 260, 'completion_tokens': 83, 'total_tokens': 343, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'service_tier': 'default', 'system_fingerprint': None}


In [68]:
output

{'id': 'chatcmpl-Cfbs61ySvgIQIbsRPWaxIiV7vbmXT',
 'object': 'chat.completion',
 'created': 1764033062,
 'model': 'gpt-4-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '"Democratic Party,85\nRepublican Party,35\nDemocrats,85\nRepublicans,35\nBlack Americans,80\nWhite Americans,75\nHispanic Americans,90\nAsian Americans,80\nMuslims,75\nChristians,70\nImmigrants,85\nGays and Lesbians,90\nJews,75\nLiberals,85\nConservatives,35\nWomen,95"',
    'refusal': None,
    'annotations': []},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 260,
  'completion_tokens': 83,
  'total_tokens': 343,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 0,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [69]:
response = output["choices"][0]["message"]['content']

In [70]:
print(response)

"Democratic Party,85
Republican Party,35
Democrats,85
Republicans,35
Black Americans,80
White Americans,75
Hispanic Americans,90
Asian Americans,80
Muslims,75
Christians,70
Immigrants,85
Gays and Lesbians,90
Jews,75
Liberals,85
Conservatives,35
Women,95"
